# Manejo de Datos en PyTorch

## Librerías

In [1]:
import csv
import functools
import gzip
import numpy as np
import pandas as pd
import torch
import json
import tempfile
import gzip
import json

from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from torch.utils.data import Dataset, DataLoader, IterableDataset

## La clase Dataset

La clase abstracta [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) es la clase base para construir un dataset de PyTorch. Cualquier dataset personalizado debe heredar de dicha clase e implementar los siguientes métodos:

- `__len__`: Para que `len(dataset)` devuelva el tamaño del conjunto de datos.
- `__getitem__`: Para soportar indexado de manera que `dataset[i]` devuelva el elemento `i`. Es común que en ciertos casos se utilice este método para levantar el dato real (e.g. una imagen) mientras que lo que se guarde en el dataset sea sólo una referencia a dicho dato (e.g. un path a la imagen). De esta manera se evita cargar muchas imágenes en memoria, haciendo que sea menos demandante a nivel RAM.

In [2]:
class IMDBReviewsDataset(Dataset):
    def __init__(self, path, transform=None):
        self.dataset = pd.read_csv(path)
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.tolist()  # Deal with list of items instead of tensor
        
        item = {
            "data": self.dataset.iloc[item]["review"],
            "target": self.dataset.iloc[item]["sentiment"]
        }

        if self.transform:
            item = self.transform(item)
        
        return item

dataset = IMDBReviewsDataset("./data/imdb_reviews.csv.gz")
print(f"Dataset loaded with {len(dataset)} elements")
print(f"Sample element:\n{dataset[0]}")

Dataset loaded with 50000 elements
Sample element:
{'data': "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><b

## Transformaciones

El ejemplo anterior nos muestra el uso básico, pero claramente no podemos pasarle eso a una red neuronal, no puede manejar texto. Es para eso que tenemos que hacer algún tipo de transformación sobre los atributos (en este caso el único atributo es el texto). 

### Normalización

En particular, como vemos en el caso anterior, el texto no está normalizado, parte de las transformaciones pueden incluir realizar algún tipo de normalización. Para eso hagamos uso de [`gensim`](https://radimrehurek.com/gensim/index.html), en particular utilizaremos el módulo [`preprocessing`](https://radimrehurek.com/gensim/parsing/preprocessing.html#module-gensim.parsing.preprocessing) que se encargará de hacer varias normalizaciones por defecto

In [3]:
class TextPreprocess:
    def __init__(self, filters=None):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _encode_target(self, target):
        return 1 if target == "positive" else 0

    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self._preprocess_string(item["data"])
        else:
            data = [self._preprocess_string(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self._encode_target(item["target"])
        else:
            target = [self._encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

preprocess = TextPreprocess()
print(preprocess(dataset[0]))

{'data': ['reviewers', 'mentioned', 'watching', 'episode', 'hooked', 'right', 'exactly', 'happened', 'thing', 'struck', 'brutality', 'unflinching', 'scenes', 'violence', 'set', 'right', 'word', 'trust', 'faint', 'hearted', 'timid', 'pulls', 'punches', 'regards', 'drugs', 'sex', 'violence', 'hardcore', 'classic', 'use', 'word', 'called', 'nickname', 'given', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'focuses', 'mainly', 'emerald', 'city', 'experimental', 'section', 'prison', 'cells', 'glass', 'fronts', 'face', 'inwards', 'privacy', 'high', 'agenda', 'city', 'home', 'aryans', 'muslims', 'gangstas', 'latinos', 'christians', 'italians', 'irish', 'scuffles', 'death', 'stares', 'dodgy', 'dealings', 'shady', 'agreements', 'far', 'away', 'main', 'appeal', 'fact', 'goes', 'shows', 'wouldn', 'dare', 'forget', 'pretty', 'pictures', 'painted', 'mainstream', 'audiences', 'forget', 'charm', 'forget', 'romance', 'mess', 'episode', 'saw', 'struck', 'nasty', 'surreal', 'couldn', 'ready',

### Conversión a vectores

Podemos continuar convertiendo el texto en una representación por vectores. Si bien hay muchas posibilidades (siendo la bolsa de palabras una de las más utilizadas), en general para Deep Learning se prefieren representaciones utilizando vectores contínuos, obtenidos por algún método del estilo de Word2Vec, Glove o FastText. Para este caso utilizaremos las representaciones de Glove de dimensión 50 que se dejaron para descargar en el [notebook 0](./0_set_up.ipynb).

In [4]:
class VectorizeText:
    def __init__(self, glove_vectors_path):
        self.glove_model = KeyedVectors.load_word2vec_format(glove_vectors_path,
                                                             binary=False, no_header=True)
        self.unkown_vector = np.random.randn(self.glove_model.vector_size)  # Random vector for unknown words
    
    def _get_vector(self, word):
        if word in self.glove_model:
            return self.glove_model[word]
        else:
            return self.unkown_vector
    
    def _get_vectors(self, sentence):
        return np.vstack([self._get_vector(word) for word in sentence])
    
    def __call__(self, item):
        review = []
        if isinstance(item["data"][0], str):
            review = self._get_vectors(item["data"])
        else:
            review = [self._get_vectors(d) for d in item["data"]]

        return {
            "data": review,
            "target": item["target"]
        }

vectorizer = VectorizeText("./data/glove.6B.50d.txt.gz")
vectorizer(preprocess(dataset[0]))

{'data': array([[-0.18105   , -0.79229999, -0.097616  , ...,  1.42859995,
         -0.032471  ,  0.47235999],
        [ 0.69395   ,  0.69261003, -0.21608   , ...,  0.2247    ,
         -0.23197   ,  0.0062523 ],
        [-0.0049087 ,  0.12611   ,  0.14056   , ..., -0.58464003,
         -0.31830999,  0.31564   ],
        ...,
        [ 0.25435999, -0.44304001, -0.12524   , ...,  0.73352998,
          0.026198  ,  0.30408001],
        [-0.058468  ,  0.019087  ,  0.089056  , ..., -0.28176001,
          0.045137  , -0.18802001],
        [ 0.14443   ,  0.39103001, -0.93454999, ..., -0.71325999,
         -0.54575998,  0.13952   ]]),
 'target': 1}

### Combinación de vectores

Si bien ahora estamos con una versión de los atributos que podría pasar por una red neuronal, hay un problema, las distintas reviews tienen largo distinto y como el algoritmo se entrena en lotes (*mini-batches*) estas requieren tener todas el mismo largo. Hay varias maneras de lidiar con esto, cada una con sus ventajas y desventajas. Dado que por ahora solo vimos perceptrón multicapa, que espera algo de tamaño fijo, una opción sencilla puede ser la de simplemente promediar los vectores de palabras.

In [5]:
class WordVectorsAverage:
    def __call__(self, item):
        if item["data"][0].ndim == 2:
            data = np.vstack([np.mean(d, axis=0) for d in item["data"]])
        else:
            data = np.mean(item["data"], axis=0)
        
        return {
            "data": data,
            "target": item["target"]
        }

vector_average = WordVectorsAverage()
vector_average(vectorizer(preprocess(dataset[0])))

{'data': array([ 1.04614057e-01,  3.10331633e-02, -1.07875598e-01, -2.80449603e-01,
         2.81640180e-01,  5.81420739e-02, -5.70409407e-02, -3.98301383e-02,
        -1.53635551e-01,  1.98029523e-01, -2.00678135e-01, -1.00644221e-01,
        -1.31576950e-01,  3.42244484e-03,  2.85826096e-01,  4.04602732e-02,
         3.21192367e-02, -9.04863921e-03, -1.20009918e-01, -2.48967753e-01,
        -6.44631160e-02,  3.04501593e-01,  1.49181718e-01,  1.91396772e-01,
         1.17939744e-01, -1.09844991e+00, -3.59090446e-01,  1.84178023e-01,
         4.26812485e-01, -1.86753290e-01,  2.02260732e+00,  1.44810874e-01,
        -4.59346306e-02, -3.99859739e-01, -7.59362426e-02,  1.00786318e-01,
        -7.28972766e-02, -1.63161299e-01, -2.06314196e-01, -2.46684950e-01,
        -1.45696838e-01,  7.73777179e-02,  4.77169609e-02,  2.68788287e-01,
         1.11495559e-01, -1.21586939e-03,  8.44473832e-03, -1.61580419e-01,
        -3.59275978e-02, -6.28511864e-02]),
 'target': 1}

### Conversión de vectores a tensores

En el paso final, debemos convertir nuestros datos de arrays de `numpy` a tensores de PyTorch.

In [6]:
class ToTensor:
    def __call__(self, item):
        """
        This espects a single array.
        """
        return {
            "data": torch.from_numpy(item["data"]),
            "target": torch.tensor(item["target"])
        }

to_tensor = ToTensor()
to_tensor(vector_average(vectorizer(preprocess(dataset[0]))))

{'data': tensor([ 1.0461e-01,  3.1033e-02, -1.0788e-01, -2.8045e-01,  2.8164e-01,
          5.8142e-02, -5.7041e-02, -3.9830e-02, -1.5364e-01,  1.9803e-01,
         -2.0068e-01, -1.0064e-01, -1.3158e-01,  3.4224e-03,  2.8583e-01,
          4.0460e-02,  3.2119e-02, -9.0486e-03, -1.2001e-01, -2.4897e-01,
         -6.4463e-02,  3.0450e-01,  1.4918e-01,  1.9140e-01,  1.1794e-01,
         -1.0984e+00, -3.5909e-01,  1.8418e-01,  4.2681e-01, -1.8675e-01,
          2.0226e+00,  1.4481e-01, -4.5935e-02, -3.9986e-01, -7.5936e-02,
          1.0079e-01, -7.2897e-02, -1.6316e-01, -2.0631e-01, -2.4668e-01,
         -1.4570e-01,  7.7378e-02,  4.7717e-02,  2.6879e-01,  1.1150e-01,
         -1.2159e-03,  8.4447e-03, -1.6158e-01, -3.5928e-02, -6.2851e-02],
        dtype=torch.float64),
 'target': tensor(1)}

### Componiendo las transformaciones

Para evitar tener que llamar a todas las funciones de transformación que querramos aplicar, para ello hacemos uso del parámetro `transform` que definimos en nuestro `Dataset` y un poco de ayuda de `functools`.

In [7]:
def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: g(f(x)), functions, lambda x: x)

dataset = IMDBReviewsDataset("./data/imdb_reviews.csv.gz",
                             transform=compose(preprocess, vectorizer, vector_average, to_tensor))
print(f"Dataset loaded with {len(dataset)} elements")
print(f"Sample element:\n{dataset[0]}")

Dataset loaded with 50000 elements
Sample element:
{'data': tensor([ 1.0461e-01,  3.1033e-02, -1.0788e-01, -2.8045e-01,  2.8164e-01,
         5.8142e-02, -5.7041e-02, -3.9830e-02, -1.5364e-01,  1.9803e-01,
        -2.0068e-01, -1.0064e-01, -1.3158e-01,  3.4224e-03,  2.8583e-01,
         4.0460e-02,  3.2119e-02, -9.0486e-03, -1.2001e-01, -2.4897e-01,
        -6.4463e-02,  3.0450e-01,  1.4918e-01,  1.9140e-01,  1.1794e-01,
        -1.0984e+00, -3.5909e-01,  1.8418e-01,  4.2681e-01, -1.8675e-01,
         2.0226e+00,  1.4481e-01, -4.5935e-02, -3.9986e-01, -7.5936e-02,
         1.0079e-01, -7.2897e-02, -1.6316e-01, -2.0631e-01, -2.4668e-01,
        -1.4570e-01,  7.7378e-02,  4.7717e-02,  2.6879e-01,  1.1150e-01,
        -1.2159e-03,  8.4447e-03, -1.6158e-01, -3.5928e-02, -6.2851e-02],
       dtype=torch.float64), 'target': tensor(1)}


### Iterando el dataset

Ya tenemos nuestro conjunto de datos con sus respectivas transformaciones. ¿Para qué nos sirve esto? Una opción es simplemente iterar en el conjunto de datos de a un elemento. Esto es sencillo, simplemente se hace a través de un `for`.

In [8]:
for idx, sample in enumerate(dataset):
    print(sample["data"])
    print(sample["target"])
    print("=" * 50)
    
    if idx == 2:
        break

tensor([ 1.0461e-01,  3.1033e-02, -1.0788e-01, -2.8045e-01,  2.8164e-01,
         5.8142e-02, -5.7041e-02, -3.9830e-02, -1.5364e-01,  1.9803e-01,
        -2.0068e-01, -1.0064e-01, -1.3158e-01,  3.4224e-03,  2.8583e-01,
         4.0460e-02,  3.2119e-02, -9.0486e-03, -1.2001e-01, -2.4897e-01,
        -6.4463e-02,  3.0450e-01,  1.4918e-01,  1.9140e-01,  1.1794e-01,
        -1.0984e+00, -3.5909e-01,  1.8418e-01,  4.2681e-01, -1.8675e-01,
         2.0226e+00,  1.4481e-01, -4.5935e-02, -3.9986e-01, -7.5936e-02,
         1.0079e-01, -7.2897e-02, -1.6316e-01, -2.0631e-01, -2.4668e-01,
        -1.4570e-01,  7.7378e-02,  4.7717e-02,  2.6879e-01,  1.1150e-01,
        -1.2159e-03,  8.4447e-03, -1.6158e-01, -3.5928e-02, -6.2851e-02],
       dtype=torch.float64)
tensor(1)
tensor([ 0.0808,  0.2028, -0.3832, -0.1337,  0.3752,  0.1792, -0.2609, -0.1271,
        -0.1494,  0.3769, -0.0622,  0.2527, -0.1175,  0.1615,  0.2949, -0.0127,
         0.1123,  0.1044, -0.1132, -0.3679,  0.0756,  0.3530,  0.0357, 

## La clase Dataloader

El problema con iterar de a un elemento es que estamos limitados al querer entrenar un modelo. Por empezar, los modelos de Deep Learning suelen ser más eficientes si se entrenan utilizando algún tipo de entrenamiento por *mini-batches*. Además, hay otras cosas como mezclar los elementos (*shuffling*) o cargar datos en paralelo vía distintos *multiprocess workers*. La clase [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) precisamente se encarga de hacer eso por nosotros:

In [9]:
dataloader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2
)

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, 
          sample_batched['data'].size(),
          sample_batched['target'].size())

    if i_batch == 2:
        print(sample_batched["data"])
        print(sample_batched["target"])
        break

0 torch.Size([4, 50]) torch.Size([4])
1 torch.Size([4, 50]) torch.Size([4])
2 torch.Size([4, 50]) torch.Size([4])
tensor([[ 1.5914e-02,  4.0997e-01, -2.3978e-01, -1.5741e-01,  3.4977e-01,
          9.8227e-02, -3.3254e-01, -7.2681e-02, -1.9208e-01,  4.0715e-01,
         -2.7488e-01,  1.7061e-01, -2.5242e-01,  1.9161e-01,  4.8638e-01,
         -1.2444e-01,  3.3351e-01,  9.0155e-02, -3.7323e-01, -4.4498e-01,
          1.5132e-01,  3.8112e-01,  9.0314e-02,  1.1210e-01,  5.2052e-01,
         -8.9548e-01, -7.0109e-01,  2.6786e-02,  3.1585e-01, -1.6027e-01,
          2.2466e+00,  3.1990e-01,  1.9264e-01, -3.2628e-01, -2.2463e-02,
          1.9148e-01, -1.3520e-02,  3.8459e-01, -1.7121e-01, -3.4719e-01,
          6.5100e-02,  2.9173e-01, -1.7915e-01,  1.0934e-02, -8.7733e-02,
          1.5263e-01,  6.0026e-02, -1.3312e-01, -1.2919e-01,  3.9585e-01],
        [ 1.8362e-01,  2.0113e-01, -1.2918e-01, -1.4424e-01,  3.1084e-01,
          1.3346e-01, -1.8407e-01, -5.6447e-02, -2.0273e-01,  3.3048e-0

## La clase IterableDataset

El método preferido para trabajar con conjuntos de datos en PyTorch es `torch.utils.data.Dataset`. En general, hacer uso inteligente del método `__getitem__`, e.g. usándolo para cargar imágenes a medida que sean necesitadas y no al instanciar el dataset, es la mejor manera de trabajar con un conjunto de datos. En particular, de esta forma es mucho más fácil hacer *shuffling* de los datos y demás. No obstante, no siempre esto es posible, muchas veces el conjunto de datos es demasiado grande para levantarlo en memoria (aunque sólo levantemos referencias). Para esos casos, PyTorch ofrece la clase [`torch.utils.data.IterableDataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset), en este caso el único método que es requerido implementar es `__iter__`.

In [10]:
class MeLiChallengeDataset(IterableDataset):
    def __init__(self, path, transform=None):
        self.dataset_path = path
        self.transform = transform

    def __iter__(self):
        with gzip.open(self.dataset_path, "rt") as fh:
            for l in fh:
                data = json.loads(l)
                item = {
                    "data": data['title'],
                    "target": data['category']
                }
                
                if self.transform:
                    yield self.transform(item)
                else:
                    yield item

dataset = MeLiChallengeDataset("./data/meli-challenge-2019/spanish.train.jsonl.gz")
dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=0)
dataiter = iter(dataloader)
print(f"Sample batch:\n{dataiter.next()}")

Sample batch:
{'data': ['Casita Muñecas Barbies Pintadas', 'Neceser Cromado Holográfico ', 'Funda Asiento A Medida D20 Chevrolet', 'Embrague Ford Focus One 1.8 8v Td (90cv) Desde 01-99'], 'target': ['DOLLHOUSES', 'TOILETRY_BAGS', 'CAR_SEAT_COVERS', 'AUTOMOTIVE_CLUTCH_KITS']}
